# Funciones de un solo sentido

## Ejercicio 1

Sea $(a_1,\ldots,a_k)$ una secuencia super-creciente de números positivos (la suma de los términos que preceden a $a_i$ es menor que $a_i$, para todo $i$). Elige $n > \sum a_i$, y $u$ un entero positivo tal que $gcd(n, u) = 1$. Define $a_i^* = ua_i \bmod n$. La función mochila (_knapsack_) asociada a $(a_1^*,\ldots,a_k^*)$ es $$f:\mathbb{Z}_2^k \rightarrow \mathbb{N}, f(x_1,\ldots,x_k) = \sum_{i=1}^k x_ia_i^*$$

Implementa esta función y su inversa, tal y como se explica en _P. J. Cameron, Notes on cryptography_. La llave pública es $((a_1,\ldots,a_k), n, u)$.

In [1]:
import AritmeticaModular as am
from random import randint

"""
    Generate a super-increasing sequence of length k.
    
    A super-increasing sequence has k elements where
    
                    a_i > sum(a_0,...,a_(i - 1))
                    
    An example: (1, 2, 4, 8, 16)
    
    k: length of the message's block
"""
def generate_super_increasing(k):
    
    # generate the first element of the sequence
    sequence = [0]*k
    sequence[0] = randint(1, k)
    
    for i in range(1, k):
        sequence[i] = sum(sequence[:i]) + randint(sequence[i-1], sequence[i-1]*2)
            
    return sequence

generate_super_increasing(8)

[2, 4, 14, 34, 118, 321, 986, 3184]

Con la función *generate_super_increasing* generamos una secuencia súper-creciente de longitud $k$, donde $k$ es el tamaño de bloque a cifrar del mensaje original.

A continuación, tenemos que tomar un número $n$ tal que $n > \sum_{i=0}^k a_i$ y un número $u$ que sea primo relativo con $n$, es decir, que $gcd(n, u) = 1$.

In [2]:
"""
    Generate the keys for the knapsack cipher.
    
    Parameters:
        block_length: length of the message's block
    Return:
        public_key: modified sequence
        private_key: original super-increasing sequence, n and u
"""

def generate_keys(block_length):
    # generate a sequence
    sequence = generate_super_increasing(block_length)
    
    n = sum(sequence) + sequence[randint(0,block_length)]
    
    found_u = False
    
    while not found_u:
        
        u = randint(1, n)
        
        if am.ext_euclides(n,u)[0] == 1: 
            found_u = True
            
    public_sequence = list(map(lambda a_i: (u*a_i) % n, sequence))
    
    return public_sequence, (sequence, n, u)

generate_keys(8)

([2011, 4022, 5764, 397, 2442, 802, 2197, 5131],
 ([3, 6, 18, 57, 176, 512, 1379, 3975], 6302, 2771))